In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import ccxt as cx
from typing import Any, Optional, List
import ccxt.pro as cxp
from ccxt.base.decimal_to_precision import ROUND_UP
import asyncio
import nest_asyncio
nest_asyncio.apply()

from ccxt.base.exchange import Exchange
from threading import Thread, Event, Lock
from queue import Queue
# from multiprocessing import Queue #as Queue

import qubx
%qubxd

%load_ext autoreload
%autoreload 2

from qubx import lookup
from qubx.core.series import TimeSeries, Bar, Trade, Quote, OHLCV
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent
from qubx.core.basics import Instrument, Position, Signal, TransactionCostsCalculator, dt_64
from qubx.impl.ccxt_connector import CCXTConnector, CtrlChannel, BinanceQV
from qubx.utils.pandas import ohlc_resample

 >  [dev] installed cython rebuilding hook

                   ╻
   ┏┓      ╻       ┃  ┏┓       Quantitative Backtesting Environment 
   ┃┃  ┓┏  ┣┓  ┏┓  ┃  ┏┛       
   ┗┻  ┗┻  ┗┛  ┗   ┃  ┗━       (c) 2024,  ver. Dev
                   ╹       



# Common

In [ ]:
lookup.instruments['KRAKEN.F:BTC/USD']

In [34]:
instr = lookup.instruments.find('BINANCE.UM', 'BTC', 'USDT')
instr

Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE.UM', base='BTC', quote='USDT', margin_symbol='USDT', min_tick=0.1, min_size_step=0.001, min_size=0.001, futures_info=FuturesInfo(contract_type='PERPETUAL', delivery_date=datetime.datetime(2100, 12, 25, 10, 0), onboard_date=datetime.datetime(2019, 9, 25, 11, 0), contract_size=1.0, maint_margin=2.5, required_margin=5.0, liquidation_fee=0.0125), _aux_instrument=None)

In [39]:
instr = lookup.instruments.find('KRAKEN', 'BTC', 'ETH')
instr

Instrument(symbol='ETH/BTC', market_type='spot', exchange='KRAKEN', base='ETH', quote='BTC', margin_symbol=None, min_tick=1e-05, min_size_step=1e-05, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [38]:
lookup.find_aux_instrument_for(instr, 'USD')

Instrument(symbol='BTC/USD', market_type='spot', exchange='KRAKEN', base='BTC', quote='USD', margin_symbol=None, min_tick=0.1, min_size_step=0.1, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [46]:
bu = cxp.binanceusdm()
bs = cxp.binance()
ks = cxp.kraken()
kf = cxp.krakenfutures()

# Get OHLC [TODO]

In [47]:
bs_markets = await bs.load_markets()
bu_markets = await bu.load_markets()
kf_markets = await kf.load_markets()
ks_markets = await ks.load_markets()

In [53]:
bu.parse_timeframe('1m')

60

In [ ]:
interval = bu.parse_timeframe('1m') * 1000
orderbook_max_levels_limit = 100
since = bu.round_timeframe('1m', bu.milliseconds(), ROUND_UP) - orderbook_max_levels_limit * interval
await bu.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=orderbook_max_levels_limit)

In [ ]:
# await bnc1.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=limit)

In [57]:
cxc = CCXT_connector('binance')

# Listen to data

In [3]:
class RunThread(Thread):
    def __init__(self, channel: CtrlChannel):
        self.result = None
        self.ch = channel
        self.loops = []
        super().__init__()

    def add(self, func, *args, **kwargs):
        self.loops.append(func(self.ch, *args, **kwargs))

    async def run_loop(self):
        self.result = await asyncio.gather(*self.loops)

    def run(self):
        self.ch.control.set()
        asyncio.run(self.run_loop())

    def stop(self):
        self.ch.control.clear()
        self.ch.queue.put((None, None)) # send sentinel


async def listen_to_trades(channel: CtrlChannel, exchange: Exchange, symbols: List[str]):
    method = 'watchTrades'
    if exchange.has[method]:
        # lock = channel.lock
        while channel.control.is_set():
            try:
                # c_time = exchange.iso8601(exchange.milliseconds())
                trades = await exchange.watch_trades_for_symbols(symbols)
                # lock.acquire()
                for trade in trades:
                    # trade = trades[-1]
                    t_ns = trade['timestamp'] * 1_000_000 # this is trade time 
                    s = trade['symbol']
                    info = trade['info']
                    price = trade['price']
                    m = info['m']
                    amnt = trade['amount'] # in base currency
                    tas = Trade(t_ns, price, amnt, int(not m), int(trade['id']))
                    channel.queue.put((s, tas))
                # lock.release()
                # print(trade, flush=True)
                # print(f"\t[{c_time}] {s} : {str(tas)}", flush=True)
            except Exception as e:
                print(type(e).__name__, str(e), flush=True)
                await exchange.close()
                raise e
    else:
        raise Exception(exchange.id + ' ' + method + ' is not supported or not implemented yet')


async def listen_to_orderbook(channel: CtrlChannel, exchange: Exchange, symbols: List[str], orderbook_max_levels_limit=5):
    lock = channel.lock
    while channel.control.is_set():
        try:
            orderbook = await exchange.watch_order_book_for_symbols(symbols, limit=orderbook_max_levels_limit)
            # - check what this time is ?
            t_ns = orderbook['timestamp'] * 1_000_000
            s = orderbook['symbol']
            b, a = orderbook['bids'], orderbook['asks']
            q = Quote(t_ns, b[0][0], a[0][0], b[0][1], a[0][1])
            # lock.acquire()
            channel.queue.put((s, q))
            # lock.release()
            # c_time = exchange.iso8601(exchange.milliseconds())
            # print(f"[{c_time}] {s} : {str(q)}", flush=True)
        except Exception as e:
            print(type(e).__name__, str(e), flush=True)
            await exchange.close()
            raise e
 

async def receiver(channel: CtrlChannel):
    sers = {}
    print("START LISTENING", flush=True)
    T, Q = 0, 0
    while channel.control.is_set():
        s, data = channel.queue.get()
        if isinstance(data, Quote):
            if not (Q % 100):
                print(s, data, flush=True)
                Q = 0
            Q += 1
        if isinstance(data, Trade):
            if s not in sers:
                sers[s] = TimeSeries(s, '1Min')
            ts = sers[s]
            ts.update(data.time, data.price)
            if not (T % 100):
                print(s, data, flush=True)
                T = 0
            T += 1
        if isinstance(data, Bar):
            # print(f"{s} {pd.Timestamp(data.time, unit='ns')}: {data}", flush=True)
            if s not in sers:
                sers[s] = {}
            sers[s][data.time] = data

    print(f"STOP LISTENING: {str(sers)}", flush=True)
    return sers 

In [4]:
def prefetch_ohlcs(exchange: Exchange, symbol: str, timeframe: str, nbarsback: int):
    assert nbarsback > 1
    start = ((pd.Timestamp('now', tz='UTC') - nbarsback * pd.Timedelta(timeframe)).asm8.item()//1000000) if nbarsback > 1 else None 
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=start, limit=nbarsback + 1)
    return ohlcv

async def listen_to_ohlcv(channel: CtrlChannel, exchange: Exchange, 
                          symbol: List[str], timeframe: str, nbarsback: int):
    # - check if we need to load initial 'snapshot'
    if nbarsback > 1:
        ohlcv = await prefetch_ohlcs(exchange, symbol, timeframe, nbarsback)
        for oh in ohlcv:
            b = Bar(oh[0] * 1_000_000, oh[1], oh[2], oh[3], oh[4], oh[6], oh[7])
            channel.queue.put((symbol, b))

    while channel.control.is_set():
        try:
            ohlcv = await exchange.watch_ohlcv(symbol, timeframe)
            for oh in ohlcv:
                b = Bar(oh[0] * 1000000, oh[1], oh[2], oh[3], oh[4], oh[6], oh[7])
                channel.queue.put((symbol, b))
        except Exception as e:
            print(type(e).__name__, str(e), flush=True)
            await exchange.close()
            raise e

In [5]:
symbs = ['1INCHUSDT', 'AGLDUSDT', 'APEUSDT', 'ARBUSDT', 'AUCTIONUSDT', 'BANDUSDT', 'BATUSDT', 'C98USDT', 'COMPUSDT', 'CYBERUSDT',
        'DYDXUSDT', 'ENSUSDT', 'FXSUSDT', 'HFTUSDT', 'ICXUSDT', 'IOSTUSDT', 'IOTAUSDT', 'IOTXUSDT', 'LDOUSDT', 'LQTYUSDT', 'LTCUSDT',
        'MASKUSDT', 'MBOXUSDT', 'MINAUSDT', 'MTLUSDT', 'OGNUSDT', 'OPUSDT', 'PEOPLEUSDT', 'POLSUSDT', 'PYRUSDT', 'QNTUSDT', 'RADUSDT',
        'RDNTUSDT', 'RIFUSDT', 'RLCUSDT', 'SEIUSDT', 'SPELLUSDT', 'SSVUSDT', 'STGUSDT', 'STXUSDT', 'SUIUSDT', 'UMAUSDT', 'WAVESUSDT']

In [6]:
chan = CtrlChannel('ticks')
reader = RunThread(chan)
reader.add(receiver)
reader.start()

START LISTENING


In [7]:
# bu = cxp.binanceusdm()
# bu = cxp.binance()
bu = BinanceQV()
# bu = getattr(cxp, 'binanceqv')()
t1 = RunThread(chan)
for s in symbs:
    t1.add(listen_to_ohlcv, bu, s, '15m', 10)

# t1.add(listen_to_ohlcv, bu, 'BTCUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'ETHUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'LTCUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'SOLUSDT', '5m', 10)
# t1.add(listen_to_trades, bu, ['BTCUSDT', 'ETHUSDT', 'SOLUSDT'])
# t1.add(listen_to_orderbook, bu, ['BTCUSDT', 'ETHUSDT', 'SOLUSDT'])
t1.start()

In [10]:
reader.stop()
t1.stop()

In [ ]:
print(len(reader.result[0]), len(symbs))
reader.result[0]

# StrategyCtx

In [232]:
from qubx import logger

class ZeroStrat(IStrategy):
    param1: int = 0

    def on_start(self, ctx: StrategyContext):
        logger.info(f"> Started with param1 = {self.param1}")

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> Optional[List[Signal]]:
        logger.info(f"{event.time} -> {event}")
        print(ctx.ohlc(ctx.instruments[0], '1Min'))
        return None

    def on_stop(self, ctx: StrategyContext):
        logger.info(f"> stopped")

In [234]:
cc = CCXTConnector('binance')
cc.time()

numpy.datetime64('2024-03-28T15:10:13.707052000')

In [ ]:
# cc.get_historical_ohlcs('BTCUSDT', '1d', 20)

In [235]:
ctx = StrategyContext(
    ZeroStrat(), dict(param1 = 1), cc, cc, 
    instruments=lookup.instruments[r'BINANCE:(BTC|ETH|LTC)USDT'],
    fees_spec='vip0_usdt', base_currency='USDT',

    md_subscription=dict(type='ohlc', timeframe='1Min', nback=10),
    trigger=dict(type='bar', delay='-1s', timeframe='1Min')
)

2024-03-28 17:10:14.549 [ ℹ️ ] Set param1 -> 1


In [236]:
ctx.start()

2024-03-28 17:10:15.516 [ ℹ️ ] Subscribing to ohlc updates using {'timeframe': '1Min', 'nback': 10} for 
	['BTCUSDT', 'ETHUSDT', 'LTCUSDT'] 
2024-03-28 17:10:15.517 [ ℹ️ ] > Started with param1 = 1
2024-03-28 17:10:15.519 [ ℹ️ ] > Data processor started
2024-03-28 17:10:15.519 [ ℹ️ ] > Market data subscribtions started
2024-03-28 17:10:15.521 [ ℹ️ ] Start processing market data


2024-03-28 17:10:59.768 [ ℹ️ ] 2024-03-28T15:10:59.768303000 -> TriggerEvent(time=numpy.datetime64('2024-03-28T15:10:59.768303000'), type='bar', instrument='LTCUSDT', data={o:94.850000 | h:94.900000 | l:94.770000 | c:94.900000 | v:124395.248620})
BTCUSDT[1Min] | 10 records
  2024-03-28T15:10:00.000000000 {o:71235.310000 | h:71264.860000 | l:71161.770000 | c:71260.000000 | v:4338877.031730}
  2024-03-28T15:09:00.000000000 {o:71238.810000 | h:71314.000000 | l:71235.300000 | c:71235.310000 | v:2245439.958868}
  2024-03-28T15:08:00.000000000 {o:71343.980000 | h:71347.060000 | l:71238.810000 | c:71238.820000 | v:2302190.839080}
   .......... 
  2024-03-28T15:03:00.000000000 {o:71454.790000 | h:71481.110000 | l:71434.480000 | c:71477.360000 | v:3083391.648339}
  2024-03-28T15:02:00.000000000 {o:71444.010000 | h:71521.000000 | l:71425.610000 | c:71454.790000 | v:5306387.934846}
  2024-03-28T15:01:00.000000000 {o:71518.000000 | h:71537.000000 | l:71432.010000 | c:71444.010000 | v:4093273.47395

In [240]:
ctx.stop()

2024-03-28 17:17:32.501 [ ℹ️ ] > stopped
2024-03-28 17:17:32.501 [ ℹ️ ] Market data processing finished


In [248]:
ohlc_resample(ctx.ohlc('BTCUSDT', '1Min').pd(), '1h')

,open,high,low,close,volume,bought_volume
timestamp,,,,,,
2024-03-28 15:00:00,71518.0,71537.0,71137.27,71185.98,5.665711e+07,2.599690e+07


In [249]:
ctx.ohlc('BTCUSDT', '1h').pd()

,open,high,low,close,volume,bought_volume
timestamp,,,,,,
2024-03-28 15:00:00,71518.0,71537.0,71137.27,71185.98,5.665711e+07,2.599690e+07


In [247]:
ctx.ohlc('BTCUSDT', '1Min')

                         open      high       low     close        volume  \
timestamp                                                                   
2024-03-28 15:01:00  71518.00  71537.00  71432.01  71444.01  4.093273e+06   
2024-03-28 15:02:00  71444.01  71521.00  71425.61  71454.79  5.306388e+06   
2024-03-28 15:03:00  71454.79  71481.11  71434.48  71477.36  3.083392e+06   
2024-03-28 15:04:00  71477.36  71494.00  71362.00  71362.00  2.398666e+06   
2024-03-28 15:05:00  71362.01  71367.91  71271.36  71337.99  5.774092e+06   
2024-03-28 15:06:00  71338.00  71353.40  71292.00  71353.40  3.389442e+06   
2024-03-28 15:07:00  71353.40  71395.39  71292.01  71343.98  3.701186e+06   
2024-03-28 15:08:00  71343.98  71347.06  71238.81  71238.82  2.302191e+06   
2024-03-28 15:09:00  71238.81  71314.00  71235.30  71235.31  2.245440e+06   
2024-03-28 15:10:00  71235.31  71270.00  71161.77  71269.99  4.451296e+06   
2024-03-28 15:11:00  71269.99  71398.00  71269.99  71373.41  3.046111e+06   

In [233]:
await cc.exch.close()

In [ ]:
# cc.close()

<hr/>

# Misc